Waze and  Graphs' Algorithms

<img src="https://pentagram-production.imgix.net/b695e6d0-864c-49f7-8290-ea973d012d0e/LogotypeWazer_Lockups_02-04.jpg?rect=194%2C0%2C3603%2C2251&w=1500&fit=crop&fm=jpg&q=70&auto=format&h=935" width="40%">

Do not start the exercise until you fully understand the submission guidelines, which can be found here.

For any material-related-questions, ask Ami. For any organization-related-questions, ask Lior.

# Waze & Dijkstra's Algorithm

We saw in class how Waze utilizes Dijkstra's Algorithm as a fundamental component of its routing system. Dijkstra's Algorithm is a graph search algorithm that finds the shortest path between two nodes in a weighted graph. In the context of Waze, cities and roads are represented as nodes and edges in a graph, respectively. Each road segment is assigned a weight that reflects the estimated travel time, considering factors such as road type, traffic conditions, and speed limits. When a user inputs a starting and destination point, Waze's implementation of Dijkstra's Algorithm efficiently explores the road network to compute the shortest route. This algorithm ensures that users receive real-time, optimal directions that consider both distance and current traffic conditions.




In [1]:
# All imports
import math
import pandas as pd

# Add more imports here if necessary:
!pip install folium
import folium
!pip install geopy
import geopy.distance
import numpy
from queue import PriorityQueue

## Question 1 (5 points):
1.1 Write a code that reads the `cities.csv` and `roads_matrix.csv` files into memory. You may choose the data structure by yourself.
* `cities.csv` file contains `V=20` cities with columns "City", "Latitude", and "Longitude".
* `roads_matrix.csv` file is a `V*V` matrix, where each cell contains a 0 (no road), a 1 (fast road) or a 2 (slow road).
1.2 Write a function `create_map` that returns a map to visualize cities and roads.


In [2]:
cities = pd.read_csv("cities.csv")
roads_matrix = pd.read_csv("roads_matrix.csv")

In [3]:
def create_map(cities, roads_matrix, show_roads=True):
    """
    Creates a map to visualize cities and roads.
    Hint: You can use the Folium library or any other library of your choice.
    Args:
    - cities (DataFrame): A DataFrame containing city data with columns 'City', 'Latitude', and 'Longitude'.
    - roads_matrix (DataFrame): A DataFrame containing road data between cities.
    - show_roads (bool): Whether to add roads to the map or not.

    Returns:
    - map: The map object.
    """
    cities_dict = create_cities_nodes(cities)

    cities_map = folium.Map(
        location=get_center_of_coords(list(cities_dict.values())),
        zoom_control=False,
        scrollWheelZoom=False,
        dragging=False
    )

    if show_roads:
      roads_dict = create_roads_dict(roads_matrix)

      for (origin_city, destinations_dict) in roads_dict.items():
        for (destination_city, road_value) in destinations_dict.items():
          add_line_to_map(
              cities_map,
               [cities_dict[origin_city],
                cities_dict[destination_city]],
              road_value
          )

    for (_, coord) in cities_dict.items():
      add_coordinate_to_map(cities_map, coord)

    cities_map.fit_bounds(cities_map.get_bounds())

    return cities_map

def create_cities_nodes(cities_df):
  cities_dict = dict()

  for i in cities.index:
      cities_dict[cities["City"][i]] = (cities["Latitude"][i], cities["Longitude"][i])

  return cities_dict

def create_roads_dict(roads_matrix_df):
  roads_dict = dict()

  for i in roads_matrix.index:
      origin_city = roads_matrix["City"][i]
      city_edges = roads_matrix[roads_matrix["City"] == origin_city]

      for (destination_city, road_data) in city_edges.items():
        if origin_city not in roads_dict:
          roads_dict[origin_city] = dict()

        road_value = int(road_data.iloc[0]) if isinstance(road_data.iloc[0], numpy.int64) else None

        if road_value:
          roads_dict[origin_city][destination_city] = road_value

  return roads_dict

def get_center_of_coords(cities_coords):
  max_lat = max(lat for lat, _ in cities_coords)
  min_lat = min(lat for lat, _ in cities_coords)
  max_long = max(lon for _, lon in cities_coords)
  min_long = min(lon for _, lon in cities_coords)

  return (max_lat + min_lat)/2, (max_long+min_long)/2

def add_coordinate_to_map(this_map, coord):
  folium.CircleMarker(location=coord,
                        radius=4,
                        color="red",
                        weight=5).add_to(this_map)

def add_line_to_map(this_map, coords, line_type):
  if line_type == 1:
    folium.PolyLine(
        coords,
        color="blue"
        ).add_to(this_map)

  if line_type == 2:
    folium.PolyLine(
        coords,
        color="green"
        ).add_to(this_map)

# Test your function:
create_map(cities, roads_matrix, show_roads=True)

## Question 2 (15 points):
Complete the class `RoadNetwork`. This class is responsible for initializing the road network, calculating distances between cities, and estimating travel times on different road types.

Note: You can change the class structure however you see fit, this is just an example to help you get started.

In [4]:
# Parameter p1 tells us how much time does it take to travel in road_type 1 in an avg pace of 25 KM/h
# Parameter p2 tells us how much time does it take to travel in road_type 2 in an avg pace of 10 KM/h
p1 = 1 / 25
p2 = 1 / 10

# We use parameters r1 and r2 to multiply the L2 distance, to get closer to a real-world road distance between the cities.
r1 = 2
r2 = 1.5

minutes_in_hour = 60

# Parameter v1 average avg pace in KM/min in on road 1
# Parameter v2 average avg pace in KM/min on road 2
v1 = (1 / p1) * (1 / minutes_in_hour)
v2 = (1 / p2) * (1 / minutes_in_hour)

average_pace_dictionary = {
    1: v1,
    2: v2
}
distance_factor_dictionary = {
    1: r1,
    2: r2
}

class RoadNetwork:
    def __init__(self, cities=cities, roads_matrix=roads_matrix):
        self._cities = create_cities_nodes(cities)
        self._roads = create_roads_dict(roads_matrix)

    def calculate_distance(self, city1, city2):
        """
        Calculates the distance (in kilometers) between two cities based on their coordinates.

        Returns:
        - distance_km (float): The distance between the two cities in kilometers.
        """
        return geopy.distance.geodesic(self._cities[city1], self._cities[city2]).km

    def calculate_travel_time(self, source_city, destination_city):
        """
        Calculates the time (in minutes) it takes to travel from source_city to destination_city, depending on road_type.

        Args:
        - source_city (str): The name of the source city.
        - destination_city (str): The name of the destination city.

        Returns:
        - travel_time (float): The travel time in minutes.
        """
        if source_city == destination_city:
          return 0

        road_type = self.road_type(source_city, destination_city)

        if not road_type:
          return math.inf

        city_arial_distance = self.calculate_distance(source_city, destination_city)
        city_approximate_distance = distance_factor_dictionary[road_type] * city_arial_distance

        return city_arial_distance / average_pace_dictionary[road_type]

    def get_adjacent_cities(self, city):
      return self._roads[city].keys()

    def road_type(self, origin, destination):
      return self._roads[origin].get(destination, 0)

    def coordinates(self, city):
      return self._cities[city]


## Question 3 (20 points):
3.1 Write your own Dijkstra's Algorithm to compute the shortest path between two cities.

3.2 Write a function `find_shortest_path` that performs the following tasks:

* Calculates and prints the estimated time of arrival (ETA) for the shortest path.
* Prints the detailed path from the starting city to the destination city.
* Optionally, displays the route on a map object for better visualization.

3.3 Test your function:
*   Find and **visualize** the shortest path from **Ashdod** to **Herzeliya**.
*   Find and **visualize** the shortest path from **Netanya** to **Tel Aviv**.
*   Choose any two cities with a minimum distance of 30 kilometers between them, and find and **visualize** the shortest path between them.



In [5]:
def dijkstra(graph: RoadNetwork, start: str, end: str):
    """
    Finds the shortest path between two cities using Dijkstra's algorithm.

    Args:
    - graph: The road network object.
    - start: The starting city.
    - end: The destination city.

    Returns:
    - path (list): The shortest path as a list of city names.
    - times (dict): The travel times to קeach city on the path.
    """
    path_dict = dict()
    times_dict = {start: 0}

    q = PriorityQueue()
    q.put((0, start))

    while not q.empty():
      (_, current_city) = q.get()

      if current_city == end:
        return _build_path_and_times(end, path_dict, times_dict)

      for next_city in graph.get_adjacent_cities(current_city):
        edge_time = graph.calculate_travel_time(current_city, next_city)
        possible_time_to_next_city = edge_time + times_dict[current_city]

        if possible_time_to_next_city < times_dict.get(next_city, math.inf):
          times_dict[next_city] = possible_time_to_next_city
          path_dict[next_city] = current_city

          q.put((possible_time_to_next_city + edge_time, next_city))

      q.task_done()

    return None


def _build_path_and_times(end_city, path_dict, times_dict):
  current_city = end_city
  reversed_path = []
  times = dict()

  while current_city:
    reversed_path.append(current_city)
    times[current_city] = times_dict[current_city]

    current_city = path_dict.get(current_city)

  return reversed_path[::-1], times

def find_shortest_path(road_network: RoadNetwork, start_city, end_city, show_on_map=False):
    """
    Prints the estimated time of arrival (ETA),
    the path from the starting city to the destination,
    and optionally displays the route on a map.

    Args:
    - start_city: The starting city.
    - end_city: The destination city.
    - show_on_map (bool): Whether to display the route on a map.

    Returns:
    - map object or None: The map object if show_on_map is True, otherwise None.
    """
    path, times = dijkstra(road_network, start_city, end_city)

    print(f"ETA from {start_city} to {end_city}: {times[end_city]:.2f} minutes")
    print("Path: " + " -> ".join(f"{city} ({times[city]:.2f} min)" for city in path))

    if show_on_map:
      return visualize_path(road_network, path)


def visualize_path(road_network: RoadNetwork, path):
  path_coordinates = [road_network.coordinates(city) for city in path]

  path_map = folium.Map(
        location=get_center_of_coords(path_coordinates),
        zoom_control=False,
        scrollWheelZoom=False,
        dragging=False
  )

  for i in range(0, len(path) - 1):
    add_line_to_map(
        path_map,
        [path_coordinates[i], path_coordinates[i+1]],
        road_network.road_type(path[i], path[i+1])
    )

  for coordinate in path_coordinates:
    add_coordinate_to_map(path_map, coordinate)

  path_map.fit_bounds(path_map.get_bounds())

  return path_map

In [6]:
network = RoadNetwork(cities, roads_matrix)
find_shortest_path(network, 'Ashdod', 'Herzeliya', show_on_map=True)

ETA from Ashdod to Herzeliya: 140.43 minutes
Path: Ashdod (0.00 min) -> Yavne (26.24 min) -> Ness Ziona (47.44 min) -> Rishon LeTsiyon (54.66 min) -> Holon (70.15 min) -> Ramat Gan (86.49 min) -> Ramat Hasharon (133.76 min) -> Herzeliya (140.43 min)
